# Create a real-time scoring endpoint with ACI

In [ ]:
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Experiment, Environment, Model
from azureml.core.compute import ComputeTarget, AksCompute

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, AksWebservice
from azureml.train.automl.run import AutoMLRun

In [ ]:
import pandas as pd
import numpy as np
import random as r
import os
import requests
import json

In [ ]:
ws = Workspace.from_config()

In [ ]:
dataset_name = "Diabetes Sample"
dataset = Dataset.get_by_name(ws, dataset_name, version='latest')


In [ ]:
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
environment = Environment.get(ws, 'AzureML-AutoML')

In [ ]:
model = Model(ws, 'Diabetes-AllData-Regression-AutoML')

In [ ]:
experiment = Experiment(ws,'Diabetes-Sample-Regression')
runID = 'AutoML_e0082cf3-ba83-401f-9220-3e51f66d6544'

In [ ]:
run = AutoMLRun(experiment, runID)

In [ ]:
best_run, fitted_model = run.get_output()

In [ ]:
os.makedirs('Real_Time_Scripts', exist_ok=True)  

In [ ]:
# Specify names for your scoring script and environment file
script_path = 'Real_Time_Scripts/Diabetes_Inference.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_path)

In [ ]:
# Give your ACI deployment a name
aci_service_name = 'diabetes-scoring-aci'

# Set your scoring script and environment file
inference_config = InferenceConfig(entry_script=script_path, environment = environment)

# Set your ACI configuration options
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'Project': 'Diabetes'}, 
                                               description = 'Diabetes Real-Time ACI Deployment')

# Deploy your model to ACI
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config,overwrite=True)
aci_service.wait_for_deployment(True)
print(aci_service.state)



# Test your ACI Deployment

In [ ]:
AGE_range = np.arange(19,79,1)
SEX_range = np.arange(1,2,1)
BMI_range = np.arange(18.0,42.2,0.1)
BP_range = np.arange(62, 133, 1)
S1_range = np.arange(97, 301, 1)
S2_range = np.arange(41.6, 242.4, 0.1)
S3_range = np.arange(22, 99, 1)
S4_range = np.arange(2, 9.09, 0.01)
S5_range = np.arange(3.258, 6.107, 0.001)
S6_range = np.arange(58, 124, 1)

In [ ]:
DiabetesList = []
columns = ['AGE','SEX','BMI','BP','S1','S2','S3','S4','S5','S6']
DiabetesDF = pd.DataFrame(columns=columns)
for i in range(0,5):
    values = [r.choice(AGE_range), r.choice(SEX_range), r.choice(BMI_range), r.choice(BP_range), r.choice(S1_range),\
              r.choice(S2_range), r.choice(S3_range), r.choice(S4_range), r.choice(S5_range), r.choice(S6_range)]
    DiabetesDict = pd.DataFrame(dict(zip(columns, values)),index=[0])
    DiabetesList.append(DiabetesDict)
DiabetesDF = DiabetesDF.append(DiabetesList,True)

In [ ]:
Dataset.Tabular.register_pandas_dataframe(DiabetesDF, datastore, 'Diabetes Scoring')

In [ ]:
test = '{"data":' + DiabetesDF.to_json(orient='records') + '}'

In [ ]:
test

In [ ]:
# Plug in the URL for your ACI web service.  You can find it easily on ML Studio by clicking endpoints.
aci_url = 'http://d076efb5-5f09-401b-aa9b-66e4a67aadab.northcentralus.azurecontainer.io/score'

# Set the content type to 'application/json'
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
response = requests.post(aci_url, test, headers=headers)
print(response.text)

# Create an AKS Cluster

In [ ]:
aks_cluster_name = 'aks-code-cluster'
vm_type = 'Standard_DS3_v2'
node_count = 3
AKS_location = 'northcentralus'


In [ ]:
prov_config = AksCompute.provisioning_configuration(vm_size = vm_type, agent_count = node_count, location = AKS_location)

In [ ]:
aks_target = ComputeTarget.create(workspace = ws, name = aks_cluster_name, provisioning_configuration = prov_config)
aks_target.wait_for_completion(show_output = True)


# Create a real-time scoring endpoint with AKS

In [ ]:
aks_cluster = AksCompute(ws, 'aks-code-cluster')

In [ ]:
aks_service_name = 'diabetes-scoring-aks' 

In [ ]:
aks_config = AksWebservice.deploy_configuration(cpu_cores = 1, 
                                                memory_gb = 1, 
                                                tags = {'Project': 'Diabetes'}, 
                                                description = 'Diabetes Real-Time ACI Deployment')

In [ ]:
aks_service = Model.deploy(ws, aks_service_name, [model], inference_config, aks_config, aks_cluster,overwrite=True)
aks_service.wait_for_deployment(show_output = True)

# Test your AKS Deployment

In [ ]:
aks_url = 'http://52.240.156.69:80/api/v1/service/diabetes-scoring-aks/score'
key = 'Ps8mFqJfcGpY4RxYAU63U7e3wgOTCKe1'
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

In [ ]:
resp = requests.post(aks_url, test, headers=headers)
print(resp.text)